In [1]:
import os
import requests
import pandas as pd

os.makedirs("data",exist_ok=True)

url = 'https://covidtracking.com/data/download/all-states-history.csv'
response = requests.get(url)

file_url = "./data/all-states-history.csv"
with open(file_url, "wb") as file:
    file.write(response.content)

df = pd.read_csv(file_url).fillna(value = 0)

In [2]:
df.head()

,date,state,death,deathConfirmed,deathIncrease,deathProbable,hospitalized,hospitalizedCumulative,hospitalizedCurrently,hospitalizedIncrease,...,totalTestResults,totalTestResultsIncrease,totalTestsAntibody,totalTestsAntigen,totalTestsPeopleAntibody,totalTestsPeopleAntigen,totalTestsPeopleViral,totalTestsPeopleViralIncrease,totalTestsViral,totalTestsViralIncrease
0,2021-03-07,AK,305.0,0.0,0,0.0,1293.0,1293.0,33.0,0,...,1731628.0,0,0.0,0.0,0.0,0.0,0.0,0,1731628.0,0
1,2021-03-07,AL,10148.0,7963.0,-1,2185.0,45976.0,45976.0,494.0,0,...,2323788.0,2347,0.0,0.0,119757.0,0.0,2323788.0,2347,0.0,0
2,2021-03-07,AR,5319.0,4308.0,22,1011.0,14926.0,14926.0,335.0,11,...,2736442.0,3380,0.0,0.0,0.0,481311.0,0.0,0,2736442.0,3380
3,2021-03-07,AS,0.0,0.0,0,0.0,0.0,0.0,0.0,0,...,2140.0,0,0.0,0.0,0.0,0.0,0.0,0,2140.0,0
4,2021-03-07,AZ,16328.0,14403.0,5,1925.0,57907.0,57907.0,963.0,44,...,7908105.0,45110,580569.0,0.0,444089.0,0.0,3842945.0,14856,7908105.0,45110


In [3]:
from dotenv import load_dotenv

load_dotenv()

True

In [4]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model='gpt-3.5-turbo-1106')
llm4o = ChatOpenAI(model='gpt-4o-mini')

In [7]:
from langchain.agents import AgentType
from langchain_experimental.agents.agent_toolkits import create_pandas_dataframe_agent

agent = create_pandas_dataframe_agent(llm=llm, df=df, verbose=True, allow_dangerous_code=True)

In [8]:
agent.invoke('how many rows are there?')



> Entering new AgentExecutor chain...
Thought: I can use the `shape` attribute of the dataframe to find out the number of rows
Action: python_repl_ast
Action Input: df.shape[0]20780I now know the final answer
Final Answer: There are 20780 rows in the dataframe.

> Finished chain.


{'input': 'how many rows are there?',
 'output': 'There are 20780 rows in the dataframe.'}

In [23]:
CSV_PROMPT_PREFIX = """
First set the pandas display options to show all the columns,
get the column names, then answer the question.
"""

CSV_PROMPT_SUFIX = """
- **ALWAYS** before giving the Final Answer, try another method.
Then reflect on the answers of the two methods you did and ask yourself
if it answers correctly the original question.
If you are not sure, try another method.
- If the methods tried do not give the same result,reflect and
try again until you have two methods that have the same result.
- If you still cannot arrive to a consistent result, say that
you are not sure of the answer.
- If you are sure of the correct answer, create a beautiful
and thorough response using Markdown.
- **DO NOT MAKE UP AN ANSWER OR USE PRIOR KNOWLEDGE,
ONLY USE THE RESULTS OF THE CALCULATIONS YOU HAVE DONE**.
- **ALWAYS**, as part of your "Final Answer", explain how you got
to the answer on a section that starts with: "\n\nExplanation:\n".
In the explanation, mention the column names that you used to get
to the final answer.
"""

In [19]:
agent = create_pandas_dataframe_agent(
    llm=llm,
    df=df,
    verbose=True,
    allow_dangerous_code=True,
    prefix=CSV_PROMPT_PREFIX,
    # suffix=CSV_PROMPT_SUFIX,
    # include_df_in_prompt=None
)

In [20]:
question = """
How may patients were hospitalized during July 2020 in Texas, and nationwide as the total of all states?
Use the hospitalizedIncrease column
"""

In [21]:
agent.invoke(question)



> Entering new AgentExecutor chain...
Thought: We need to filter the data for July 2020 and then sum up the hospitalizedIncrease column for Texas and all states.
Action: python_repl_ast
Action Input: df['date'] = pd.to_datetime(df['date'])NameError: name 'pd' is not definedI need to import the pandas library before using it
Action: python_repl_ast
Action Input: import pandas as pdNow I can use the pandas library. Let's filter the data for July 2020 and then sum up the hospitalizedIncrease column for Texas and all states.
Action: python_repl_ast
Action Input: df['date'] = pd.to_datetime(df['date'])I made a mistake, I need to filter the data for July 2020 first before calculating the sum of hospitalizedIncrease for Texas and nationwide.
Action: python_repl_ast
Action Input: df['date'] = pd.to_datetime(df['date'])I made a mistake again. I need to filter the data for July 2020 first.
Action: python_repl_ast
Action Input: df['date'] = pd.to_datetime(df['date'])I should filter the data for

{'input': 'How may patients were hospitalized during July 2020 in Texas, and nationwide as the total of all states?\nUse the hospitalizedIncrease column',
 'output': 'In July 2020, there were 20371 patients hospitalized in Texas, and 212740 patients hospitalized nationwide.'}

In [27]:
agent = create_pandas_dataframe_agent(
    llm=llm,
    df=df,
    verbose=True,
    allow_dangerous_code=True,
    handle_parsing_errors=True,
    agent_executor_kwargs={'handle_parsing_errors': True}
)

/home/ubuntu/.local/share/virtualenvs/llm-agent-demos-bAztRnc3/lib/python3.11/site-packages/langchain_experimental/agents/agent_toolkits/pandas/base.py:283: UserWarning: Received additional kwargs {'handle_parsing_errors': True} which are no longer supported.
  warnings.warn(


In [28]:
agent.invoke(CSV_PROMPT_PREFIX + question + CSV_PROMPT_SUFIX)



> Entering new AgentExecutor chain...
Thought: I need to set pandas display options to show all the columns and get the column names to answer the question.
Action: python_repl_ast
Action Input: 
```python
import pandas as pd
pd.set_option('display.max_columns', None)
print(df.columns)
```Index(['date', 'state', 'death', 'deathConfirmed', 'deathIncrease',
       'deathProbable', 'hospitalized', 'hospitalizedCumulative',
       'hospitalizedCurrently', 'hospitalizedIncrease', 'inIcuCumulative',
       'inIcuCurrently', 'negative', 'negativeIncrease',
       'negativeTestsAntibody', 'negativeTestsPeopleAntibody',
       'negativeTestsViral', 'onVentilatorCumulative', 'onVentilatorCurrently',
       'positive', 'positiveCasesViral', 'positiveIncrease', 'positiveScore',
       'positiveTestsAntibody', 'positiveTestsAntigen',
       'positiveTestsPeopleAntibody', 'positiveTestsPeopleAntigen',
       'positiveTestsViral', 'recovered', 'totalTestEncountersViral',
       'totalTestEncounters

{'input': '\nFirst set the pandas display options to show all the columns,\nget the column names, then answer the question.\nHow may patients were hospitalized during July 2020 in Texas, and nationwide as the total of all states?\nUse the hospitalizedIncrease column\n- **ALWAYS** before giving the Final Answer, try another method.\nThen reflect on the answers of the two methods you did and ask yourself\nif it answers correctly the original question.\nIf you are not sure, try another method.\n- If the methods tried do not give the same result,reflect and\ntry again until you have two methods that have the same result.\n- If you still cannot arrive to a consistent result, say that\nyou are not sure of the answer.\n- If you are sure of the correct answer, create a beautiful\nand thorough response using Markdown.\n- **DO NOT MAKE UP AN ANSWER OR USE PRIOR KNOWLEDGE,\nONLY USE THE RESULTS OF THE CALCULATIONS YOU HAVE DONE**.\n- **ALWAYS**, as part of your "Final Answer", explain how you got